In [120]:
import spotipy
import pandas as pd
from datetime import date 
from datetime import datetime
from spotipy.oauth2 import SpotifyClientCredentials
import sqlalchemy as sal

In [121]:
# Connect to Spotify
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials('<enter client id>','<enter secret id>'))

In [122]:
# Extract Phase
# Get top 50 songs playlist
my_playlist = spotify.playlist_tracks(playlist_id='37i9dQZEVXbLZ52XmnySJg')

In [123]:
# Transform Phase
# Get required details from data
track_names = []
track_ids = []
track_duration_minutes = []
track_preview_url = []
track_artists = []
track_popularity = []
track_release_date = []
song_ranks = []
cnt = 1
for items in my_playlist['items']:
    song_ranks.append(cnt)
    track_names.append(items['track']['name'])
    track_ids.append(items['track']['id'])
    track_duration_minutes.append(round(items['track']['duration_ms']/60000))
    track_preview_url.append(items['track']['preview_url'])
    track_popularity.append(items['track']['popularity'])
    track_release_date.append(items['track']['album']['release_date'])
    artist_names = ''
    for artist in items['track']['artists']:
        artist_names += artist['name'] + ';'
    track_artists.append(artist_names)
    cnt += 1

In [124]:
# Create final dataset
spotify_track = {
    'report_date': date.today(),
    'track_name': track_names,
    'track_id': track_ids,
    'track_duration_minutes': track_duration_minutes,
    'track_preview_url':track_preview_url,
    'track_artists':track_artists,
    'track_popularity':track_popularity,
    'track_release_date':track_release_date,
    'song_rank': song_ranks,
    'record_added_date':datetime.now(),
    'record_updated_date': datetime.now()
}

# Creating Data frame
df_spotify = pd.DataFrame(spotify_track)

In [128]:
# Load Phase (Load dataset to DB)
engine = sal.create_engine('mssql://W10DLQCCK3\SQLEXPRESS/Python_ETL?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

def load_to_db(df_spotify):
    df_spotify.to_sql('spotify_top_50_songs', con=conn, index=False, if_exists='append')
    conn.commit()

load_to_db(df_spotify)